In [1]:
import yfinance as yf, pandas as pd, shutil, os, time, glob
import numpy as np
import requests
from get_all_tickers import get_tickers as gt
from statistics import mean
import talib
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
class Analysis:

    def __init__(self,stock,dtype):

        self.stock = stock
        self.dtype = dtype
        self.prices = []
    

    def stock_data(self):
        if self.dtype == 'm':
            #try except 
            self.hist_data = pd.read_csv("/Users/ks/bot/Pynance/yfinance/stock_data_min/"+self.stock.upper()+".csv")
        elif self.dtype == 'h':
            pass
        elif self.dtype == 'd':
            self.hist_data = pd.read_csv("/Users/ks/bot/Pynance/yfinance/stock_data_daily/"+self.stock.upper()+".csv")

        c = 0
        while c < len(self.hist_data):
            if self.hist_data.iloc[c,4] > float(2.00):
                self.prices.append(self.hist_data.iloc[c,4])
                c += 1


    def rsi_indicator(self, time_period=14):
        self.time_period = time_period

        i = 0
        self.up_prices=[]
        self.down_prices=[]
        while i < len(self.prices):
            if i == 0:
                self.up_prices.append(0)
                self.down_prices.append(0)
            else:
                if (self.prices[i]-self.prices[i-1])>0:
                    self.up_prices.append(self.prices[i]-self.prices[i-1])
                    self.down_prices.append(0)
                else:
                    self.down_prices.append(self.prices[i]-self.prices[i-1])
                    self.up_prices.append(0)
            i += 1

        x = 0
        self.avg_gain = []
        self.avg_loss = []
        while x < len(self.up_prices):
            if x < self.time_period+1:
                self.avg_gain.append(0)
                self.avg_loss.append(0)
            else:
                sum_gain = 0
                sum_loss = 0
                y = x-self.time_period
                while y < x:
                    sum_gain += self.up_prices[y]
                    sum_loss += self.down_prices[y]
                    y += 1
                self.avg_gain.append(sum_gain/self.time_period)
                self.avg_loss.append(abs(sum_loss/self.time_period))
            x += 1

        p = 0
        self.rs = []
        self.rsi = []
        # Loop to calculate RSI and RS
        while p < len(self.prices):
            if p < self.time_period+1:
                self.rs.append(0)
                self.rsi.append(0)
            else:
                rs_value = (self.avg_gain[p]/self.avg_loss[p])
                self.rs.append(self.rs_value)
                self.rsi.append(100 - (100/(1+rs_value)))
            p += 1

        #RSI_smoothed calculation
        self.rsi_smoothed = talib.RSI(np.asarray(self.prices), timeperiod=time_period)


    def bbands_indicator(self, time_period=21):
        self.time_period = time_period
        
        self.upperband, self.middleband, self.lowerband = talib.BBANDS(np.asarray(self.prices), timeperiod=self.time_period, nbdevup=2, nbdevdn=2, matype=0)
        self.upperband_ema, self.middleband_ema, self.lowerband_ema = talib.BBANDS(np.asarray(self.prices), timeperiod=self.time_period, nbdevup=2, nbdevdn=2, matype=talib.MA_Type.T3)

    def plots():
        
        
